In [1]:
import glob
import os
from osgeo import gdal, osr
import numpy as np
import pandas as pd

In [2]:
# Glob together all of the Snow Fraction datasets.
albedo = glob.glob('*.h5')
albedo

['SierraAlbedo2001.h5',
 'SierraAlbedo2002.h5',
 'SierraAlbedo2003.h5',
 'SierraAlbedo2004.h5',
 'SierraAlbedo2005.h5',
 'SierraAlbedo2006.h5',
 'SierraAlbedo2007.h5',
 'SierraAlbedo2008.h5',
 'SierraAlbedo2009.h5',
 'SierraAlbedo2010.h5',
 'SierraAlbedo2011.h5',
 'SierraAlbedo2012.h5',
 'SierraAlbedo2013.h5',
 'SierraAlbedo2014.h5',
 'SierraAlbedo2015.h5',
 'SierraAlbedo2016.h5',
 'SierraAlbedo2017.h5',
 'SierraAlbedo2018.h5',
 'SierraAlbedo2019.h5']

In [3]:
# Get subdatasets of first snow fraction dataset ('SierraAlbedo2001.h5').
dataset = gdal.Open(albedo[0], gdal.GA_ReadOnly)


#Changes the selected dataset into an array.

albedo_array = dataset.ReadAsArray()

albedo_float = albedo_array.astype('float')
albedo_float[albedo_float == 65535] = np.nan
# Review whether to make snow 0 or na 
albedo_float[albedo_float == 0] = np.nan
albedo_transposed = np.transpose(albedo_float)

#Sample data 
albedo_subset = albedo_transposed[:,:,160]
albedo_permuted = np.transpose(albedo_float, (0, 2, 1))

In [5]:
# x dimension of array
xdim = albedo_array.shape[1]
# y dimension of array
ydim = albedo_array.shape[2]
# Projection data of sample GeoTiff
projection = 'PROJCS["Albers Conical Equal Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
# transformation data of array
# Pull refrencing matrix from h5 file.
ref_matrix_meta = dataset.GetMetadata()['Grid_MODIS_GRID_500m_ReferencingMatrix'].split()
referencing_matrix = [int(ref_matrix_meta[2]), int(ref_matrix_meta[1]), int(ref_matrix_meta[0]), int(ref_matrix_meta[5]), int(ref_matrix_meta[4]), int(ref_matrix_meta[3])]

## Convert Single Single H5 Layer to Geotiff

In [7]:
def SingleGeotiff(raster_name, data, height, width, geotransform, wkt):
    
    driver = gdal.GetDriverByName('GTiff')

    dataset = driver.Create(
        raster_name,
        width,
        height,
        1,
        gdal.GDT_Float32)

    dataset.SetGeoTransform((
     geotransform))

    dataset.SetProjection(wkt)
    dataset.GetRasterBand(1).WriteArray(data)
    dataset.FlushCache()  # Write to disk.
    return dataset, dataset.GetRasterBand(1) 

In [20]:
SingleGeotiff('test_file.tif', albedo_subset, ydim, xdim, referencing_matrix, projection)

(<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000015DEE2E53F0> >,
 <osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x0000015DEE2A29C0> >)

## Convert Stacked H5 File into N Single Geotiffs

In [86]:
path = 'tif/2001'
for i in range(len(albedo_float)):
    dest = (str(i + 1) + '2001.tif') 
    name = os.path.join(path, dest)
    data = albedo_transposed[:,:,i]
    SingleGeotiff(name, data, ydim, xdim, referencing_matrix, projection)

In [ ]:
path = 'tif/2001'
dest = 'sample'
name = os.path.join(path, dest)
name

'tif/2001\\sample'

## Convert Stacked H5 File to Geotiff

In [7]:
def StackedGeotiff(name, array, geo_transform, projection):
    
    driver = gdal.GetDriverByName('GTiff')

    DataSet = driver.Create(name, array.shape[2], array.shape[1], array.shape[0], gdal.GDT_Float32)
    DataSet.SetGeoTransform(geo_transform)
    DataSet.SetProjection(projection)
    for i, image in enumerate(array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
    DataSet.FlushCache()
    return name

In [8]:
StackedGeotiff('sample_stack.tif', albedo_permuted, referencing_matrix, projection)

'sample_stack10.tif'

## Transform Geotiff to WGS84 

In [23]:
# Change the following variables to the file you want to convert (inputfile)
# and what you want to name your output file (outputfile)
inputfile = "test_file.tif"
outputfile = "wgs_test.tif"
#Do not change the following line, it will reproject the geotiff file
ds = gdal.Warp(outputfile, inputfile, dstSRS="+proj=longlat +datum=WGS84 +no_defs", dstNodata = 0)